# Bike Position - Video Analysis Tool

***
With this video analysis tool, the user can explore a recorded video of an athlete pedalling on a (triathlon) bike and perform the following actions:

* move from frame to frame
* draw lines and points to measure important angles
* save frames with the above-mentioned drawings

Here is an example of the output of this script, in which the user has measured the hip angle of the portraid athlete:

![Example Output](docs/example_output.png)

The tool is still in it's rudimentary form, the following features are currently envisioned and will be implemented over time:
* automatic recognition of important points in the frame (foot, ankle, knee, hip, shoulder, elbow and hands)
* automatic and continuous measurement of multiple angles
* time series of tracked angles
* integration of power and cadence data from .fit activity files

The tool is stored at the following link: https://github.com/matteobe/BikePosition, in case you want to contribute, reach out on GitHub. 

Enjoy!

## Import Packages

In [62]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import io

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from PIL import Image
from ipycanvas import MultiCanvas

# Enable ipython widgets
import ipywidgets as widgets

# Display matplotlib images directly in the Jupyter notebook
%matplotlib inline

## Define input video and frame output

In [223]:
# Define input video
input_video = 'Videos/DSC_3863.mov'

# Define target name for frame output and extension type
frame_output = 'Frames/DSC_3863_'
frame_size = (1280,720)

# Start saved frames counter and video start
current_frame = 0
saved_frames = 1

## Helper Functions

Functions defined to retrieve and update frames.

In [224]:
# Function to extract frame from video, so that it can be displayed in a widget
def get_frame():
    # Define global variables that are accessible
    global current_frame, video, video_fps, frame_size, frame_format
    
    # Extract the frame
    frame_img = video.get_frame(current_frame / video_fps)
    img = Image.fromarray(frame_img,'RGB')
    img_resize = img.resize(frame_size)
    buf = io.BytesIO()
    img_resize.save(buf, format=frame_format)
    return buf.getvalue()

def update_frame():
    # Retrieve new frame and replace image in image widget
    frame_img = get_frame()
    image_widget.value = frame_img
    
    # Redraw canvas element containing the image widget
    update_canvas_image()
    
def save_frame():
    # Define global variables that are accessible
    global mcanvas, saved_frames
    
    # Change canvas properties to synchronize data
    mcanvas.sync_image_data = True
    for i in range(8):
        mcanvas[i].sync_image_data = True
    
    # Update the image and drawings
    update_canvas_image()
    update_drawings(True)
    
    # Storage of frame is done by callback function of canvas
    
    # Reset sync_image_data setting
    mcanvas.sync_image_data = False
    for i in range(8):
        mcanvas[i].sync_image_data = False
    
    # Increase the saved_frames counter
    saved_frames += 1


# Read in the first frame of the video and retrieve video properties
video = VideoFileClip(input_video)
video_fps = video.fps
video_frames = video.reader.nframes
frame_img = get_frame()

## Widgets Functions

Below we define different widgets used to implement the video reading and storing of frames.

In [225]:
# Define the image widget and the image replacement function
image_widget = widgets.Image(value=frame_img)


# Define slider for quick move-around in the video
slider_time = widgets.IntSlider(
    value=current_frame,
    min=0,
    max=video_frames,
    step=1,
    description='Frame slider:',
    orientation='horizontal',
    continuous_update=False,
    readout=False,
    readout_format='d')

def time_slider_moved(change):
    # Define global variables accessible within function
    global current_frame
    
    # Change the value of the current frame variable and replace image
    current_frame = change['new']
    update_frame()

slider_time.observe(time_slider_moved, names='value')


# Define the next frame button
button_nextFrame = widgets.Button(description="Next frame")

def button_nextFrame_clicked(b):
    # Define global variables accessible within function
    global current_frame, video_frames
    
    # Increase current frame
    if current_frame + 1 < video_frames:
        current_frame += 1
    
    # Retrieve new frame and replace image in image widget
    update_frame()

button_nextFrame.on_click(button_nextFrame_clicked)
    

# Define the save frame button
button_saveFrame = widgets.Button(description="Save frame")

def button_saveFrame_clicked(b):
    # Save image
    save_frame()
    
button_saveFrame.on_click(button_saveFrame_clicked)


# Define the previous frame button
button_previousFrame = widgets.Button(description="Previous frame")

def button_previousFrame_clicked(b):
    # Define global variables accessible within function
    global current_frame
    
    # Decrease current frame
    if current_frame - 1 >= 0:
        current_frame -= 1
    
    # Retrieve new frame and replace image in image widget
    update_frame()

button_previousFrame.on_click(button_previousFrame_clicked)

# Define horizontal box with the buttons
buttons_box = widgets.HBox([slider_time,button_previousFrame,button_saveFrame,button_nextFrame])

## Points tracking widgets

Functions definition for sliders and drawing tools used to track, move and draw points, lines and angles on the canvas image.

In [226]:
# Resize image to fit on screen
resize_factor = 1280/frame_size[0]*0.7
c_width = int(frame_size[0]*resize_factor)
c_height = int(frame_size[1]*resize_factor)


# Horizontal position slider
slider_horizontal = widgets.IntSlider(
    value=0,
    min=0,
    max=c_width,
    step=1,
    orientation='horizontal',
    continuous_update=True,
    readout=False,
    readout_format='d',
    layout=widgets.Layout(width=str(c_width)+'px',margin='0px 0px 0px 0px', padding='0px 0px 0px 0px'))

# Horizontal slider position change function
def horizontal_slider_moved(change):
    # Define global variables accessible within function
    global points_name, points_x, select_point
    
    # Retrieve index of current point
    indx = points_name.index(select_point.value)
    points_x[indx] = change['new']
    
    # Update the drawings
    update_drawings()

slider_horizontal.observe(horizontal_slider_moved, names='value')


# Vertical position slider
slider_vertical = widgets.IntSlider(
    value=c_height,
    min=0,
    max=c_height,
    step=1,
    orientation='vertical',
    continuous_update=True,
    readout=False,
    readout_format='d',
    layout=widgets.Layout(height=str(c_height)+'px',margin='0px 0px 0px 0px', padding='0px 0px 0px 0px'))

# Vertical slider position change function
def vertical_slider_moved(change):
    # Define global variables accessible within function
    global points_name, points_y, select_point, c_height
    
    # Retrieve index of current point
    indx = points_name.index(select_point.value)
    points_y[indx] = c_height - change['new']
    
    # Update the drawings
    update_drawings()

slider_vertical.observe(vertical_slider_moved, names='value')

In [227]:
# Define points names and coordinates
points_name = ['orange','cyan','lime']
points_x = [0,0,c_width]
points_y = [c_height,0,0]

# Define buttons to select point that will be moved
select_point = widgets.ToggleButtons(
    options=points_name,
    value=points_name[1],
    description='Active point:',
    button_style='',
    tooltips=[s+' point' for s in points_name])

# Function to update sliders based on point chosen
def newly_selected_point(change):
    # Define global variables
    global points_name, points_x, points_y, slider_horizontal, slider_vertical, c_height
    
    # Retrieve new point value
    point = change['new']
    
    # Get relevant coordinates
    indx = points_name.index(point)
    x = points_x[indx]
    y = points_y[indx]
    
    # Update sliders values
    slider_horizontal.value = x
    slider_vertical.value = c_height - y
    
# Observe point selector and call function in case selected point changes
select_point.observe(newly_selected_point, names='value')

## Canvas widgets

Functions definition for drawing points and lines on the image interactively.

Guidelines on how to use the `ipycanvas` package can be found [here](https://readthedocs.org/projects/ipycanvas/downloads/pdf/latest/).

In [228]:
# Define a multilayer canvas with 7 layers (background, 1 angle, 1 text, 2 lines, 3 points)
mcanvas = MultiCanvas(8, width=c_width, height=c_height, sync_image_data=False)

# Callback function and listener to save canvas image data
def save_canvas_to_file(*args,**kwargs):
    # Define access to global variables
    global frame_output, saved_frames
    
    # Save canvas to file
    mcanvas.to_file(frame_output+str(saved_frames).zfill(3)+'.png')
    
# Canvas listener
mcanvas.observe(save_canvas_to_file,'image_data')
    

# Create a box where to store and show the canvas
image_box = widgets.HBox([mcanvas,slider_vertical])
canvas_box = widgets.VBox([slider_horizontal,image_box])

# Define canvas characteristics
# Angle
mcanvas[1].fill_style = 'yellow'
mcanvas[1].stroke_style = 'yellow'
mcanvas[1].line_width = 2.0
mcanvas[1].global_alpha = 0.6

# Text
mcanvas[2].font = "16px sans-serif"
mcanvas[2].fill_style = 'black'
mcanvas[2].text_align = 'center'
mcanvas[2].text_baseline = 'middle'

# Lines
for i in [3,4]:
    mcanvas[i].line_width = 3.0
    mcanvas[i].line_cap = 'round'
    mcanvas[i].stroke_style = 'magenta'
    
# Points
for i in [5,6,7]:
    mcanvas[i].fill_style = points_name[i-5]

    
# Function for updating the image in the canvas
def update_canvas_image():
    # Define global variables necessary for function
    global mcanvas, image_widget, c_width, c_height
    
    # Clear canvas and redraw
    mcanvas[0].draw_image(image_widget,x=0,y=0,width=c_width,height=c_height)
    
    
# Define function grouping all drawings
def update_drawings(draw_all=False):
    # Draw angle
    draw_angle()
    # Draw lines
    draw_lines(draw_all)
    # Draw points
    draw_points(draw_all)


# Functions for drawing the line segments between the points
def draw_lines(draw_all=False):
    # Define global variables used to draw the lines
    global mcanvas, points_name, points_x, points_y, select_point
    
    # Local variables 
    c_offset = 3
    
    # Retrieve point that is being moved, thus influencing the line segment
    indx = points_name.index(select_point.value)
    
    # Check if we need to draw all points and create index of all canvases to be drawn on
    if draw_all or indx==1:
        c_indx = [c_offset,c_offset+1]
    elif indx==0:
        c_indx = [c_offset]
    else:
        c_indx = [c_offset+1]
    
    # Redraw the lines (canvas 1: point1-point2, canvas 2: point2-point3)
    for i in c_indx:
        mcanvas[i].clear()
        mcanvas[i].begin_path()
        mcanvas[i].move_to(points_x[i-c_offset],points_y[i-c_offset])
        mcanvas[i].line_to(points_x[i-c_offset+1],points_y[i-c_offset+1])
        mcanvas[i].stroke()
        
        
# Function for drawing points
def draw_points(draw_all=False):
    # Define global variables used to draw the points
    global mcanvas, points_name, points_x, points_y, select_point
    
    # Define local variables
    radius = 5
    c_offset = 5
    
    # Check if we need to draw all the points
    if draw_all:
        indx = [0,1,2]
    else:
        # Retrieve point that is being moved, thus influencing the line segment
        indx = [points_name.index(select_point.value)]
    
    # Draw the required points
    for i in indx:
        mcanvas[i+c_offset].clear()
        mcanvas[i+c_offset].fill_arc(points_x[i],points_y[i],radius,0,2*math.pi)
    
    
# Function for drawing the angle between the two lines
def draw_angle():
    # Define global variables used to draw the angle
    global mcanvas, points_x, points_y
    
    # Define local variables
    radius = 60
    x = points_x[1]
    y = points_y[1]
    
    # Calculate start and end angle
    start_angle = get_angle(points_x[2]-x,points_y[2]-y)
    end_angle = get_angle(points_x[0]-x,points_y[0]-y)
    
    # Check that end angle is larger than start angle by adding 2pi
    if end_angle < start_angle:
        end_angle += 2*math.pi
    
    # Check if the delta angle is larger than pi, in which case
    if end_angle - start_angle > math.pi:
        s = start_angle
        start_angle = end_angle - 2*math.pi
        end_angle = s
        
    # Calculate delta angle
    delta_angle = end_angle - start_angle
    
    # Draw the arc and the triangle to get a filled out arc
    c = mcanvas[1]
    c.clear()
    c.begin_path()
    c.move_to(x+radius*math.cos(start_angle),y+radius*math.sin(start_angle))
    c.line_to(x,y)
    c.line_to(x+radius*math.cos(end_angle),y+radius*math.sin(end_angle))
    c.fill()
    c.fill_arc(x,y,radius,start_angle,end_angle)
    
    # Draw the text with the angle
    mcanvas[2].clear()
    mcanvas[2].fill_text(str(int(delta_angle/math.pi*180))+"°",
                         x+2/3*radius*math.cos(start_angle+delta_angle/2), 
                         y+2/3*radius*math.sin(start_angle+delta_angle/2))
    
    
# Function for properly returning the correct angle
def get_angle(dx,dy):

    # Go through the different cases
    if dx > 0 and dy > 0: 
        # Between 0 and pi/2
        return math.atan(dy/dx)
    elif dx > 0 and dy < 0:
        # Between 3/4*pi and 2*pi
        return 2*math.pi + math.atan(dy/dx)
    elif dx < 0 and dy > 0:
        # Between pi/2 and pi
        return math.pi + math.atan(dy/dx)
    elif dx < 0 and dy < 0:
        # Between pi and 3/4*pi
        return math.pi + math.atan(dy/dx)
    elif dx == 0 and dy > 0:
        # pi/2
        return 1/2*math.pi
    elif dx == 0 and dy < 0:
        # 3/4*pi
        return 3/4*math.pi
    elif dx < 0 and dy == 0:
        # pi
        return math.pi
    else:
        return 0

        
# Draw images, lines, points in the canvas
update_canvas_image()
update_drawings(True)

## User interface

Please run the following cell, in order to access the full functionality of this tool. 

For informatation, the two sliders on the sides of the image are for moving the active points on the image to a desired point, in order to measure posture angles.

In [229]:
# Display all widgets for user to use
display(select_point)
display(canvas_box)
display(buttons_box)

ToggleButtons(description='Active point:', index=1, options=('orange', 'cyan', 'lime'), tooltips=('orange poin…